<a href="https://colab.research.google.com/github/Pathin220/4105_Hw6/blob/main/Hw6_P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error
import tensorflow
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

In [90]:
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/My Drive/Machine Learning/Housing.csv'
housing = pd.DataFrame(pd.read_csv(file_path))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [91]:
varlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']

# Defining the map function
def mapping(x):
    return x.map({'yes': 1, 'no': 0, 'furnished':  1, 'semi-furnished':  0, 'unfurnished':  -1})

housing[varlist] = housing[varlist].apply(mapping)

In [92]:
#Splitting the Dataset
x_train, x_val = train_test_split(housing, train_size = 0.8, test_size = 0.2, random_state = 0)

#Splitting Variables
y_train = x_train.pop("price")
y_val = x_val.pop("price")
y_train = y_train.values.reshape(-1, 1)
y_val = y_val.values.reshape(-1, 1)


In [93]:
standard = StandardScaler()
vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'mainroad', 'guestroom', 'basement', 'hotwaterheating',
         'airconditioning', 'prefarea', 'furnishingstatus']

In [94]:
x1_train_stan = x_train
x1_train_stan[vars] = standard.fit_transform(x1_train_stan[vars])

x1_val_stan = x_val
x1_val_stan[vars] = standard.fit_transform(x1_val_stan[vars])

In [95]:
#Stacking them there inputs
x_train_stan_numpy = np.c_[np.ones((len(y_train), 1)), x1_train_stan[vars]]
x_val_stan_numpy = np.c_[np.ones((len(y_val), 1)), x1_val_stan[vars]]


x_train_ten = torch.tensor(x_train_stan_numpy, dtype=torch.float)
x_val_ten = torch.tensor(x_val_stan_numpy, dtype=torch.float)
y_train_ten = torch.tensor(y_train, dtype=torch.float)
y_val_ten = torch.tensor(y_val, dtype=torch.float)

In [96]:
#1a. Develop a Fully Connected Neural Network with only one hidden layer (size of 32) to predict the housing value for the housing dataset.
#Make sure to include all input features. Compare your training loss value and validation results against the linear regression you implemented in Homework 5.
#Can you compare your model complexity (number of trainable parameters) against linear regression? Note: Perform 20%, and 80% split for training and validation.

In [97]:
#one hidden layer
seq_model_single = nn.Sequential(
            nn.Linear(13, 32), # <1>
            nn.Tanh(),
            nn.Linear(32, 1)) # <2>
seq_model_single

Sequential(
  (0): Linear(in_features=13, out_features=32, bias=True)
  (1): Tanh()
  (2): Linear(in_features=32, out_features=1, bias=True)
)

In [98]:
#training loop
def training_loop(n_epochs, optimizer, model, loss_fn, x_train, x_val,
                  y_train, y_val):
    for epoch in range(1, n_epochs + 1):
        p_train = model(x_train) # <1>
        loss_train = loss_fn(p_train, y_train,)

        p_val = model(x_val) # <1>
        loss_val = loss_fn(p_val, y_val)

        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 250 == 0:
            print(f"Epoch: {epoch}, Training loss: {loss_train.item():.4f},"
                  f" Validation loss: {loss_val.item():.4f}")

In [99]:
optimizer = optim.SGD(seq_model_single.parameters(), lr=1e-2)

training_loop(
    n_epochs = 5000,
    optimizer=optimizer,
    model = seq_model_single,
    loss_fn = nn.MSELoss(),
    x_train = x_train_ten,
    x_val = x_val_ten,
    y_train = y_train_ten,
    y_val = y_val_ten
)

Epoch: 1, Training loss: 26469979914240.0000, Validation loss: 25189242896384.0000
Epoch: 250, Training loss: 3640052154368.0000, Validation loss: 2884621565952.0000
Epoch: 500, Training loss: 3639144873984.0000, Validation loss: 2884644372480.0000
Epoch: 750, Training loss: 3638315188224.0000, Validation loss: 2884665868288.0000
Epoch: 1000, Training loss: 3637555757056.0000, Validation loss: 2884686577664.0000
Epoch: 1250, Training loss: 3636861075456.0000, Validation loss: 2884706500608.0000
Epoch: 1500, Training loss: 3636225114112.0000, Validation loss: 2884725637120.0000
Epoch: 1750, Training loss: 3635643678720.0000, Validation loss: 2884743725056.0000
Epoch: 2000, Training loss: 3635111002112.0000, Validation loss: 2884761288704.0000
Epoch: 2250, Training loss: 3634624200704.0000, Validation loss: 2884778590208.0000
Epoch: 2500, Training loss: 3634178293760.0000, Validation loss: 2884794580992.0000
Epoch: 2750, Training loss: 3633770659840.0000, Validation loss: 2884810571776.0

In [100]:
#1b. We will increase the network complexity by adding two additional hidden layers, the hidden layers overall.
#My suggestions for the size of layers are: 32, 64, 16, respectively.
#Please redesign the network and compare your training loss value and validation results against the linear regression you implemented in Homework 5 and Problem 1.a.
#Can you compare your model complexity? Note: Use the same 20%, and 80% split for training and validation.

In [101]:
#three hidden layer
seq_model_multiple = nn.Sequential(
            nn.Linear(13, 32), # <1>
            nn.Tanh(),
            nn.Linear(32, 64), # <2>
            nn.Tanh(),
            nn.Linear(64, 16), # <3>
            nn.Tanh(),
            nn.Linear(16,1))
seq_model_multiple

Sequential(
  (0): Linear(in_features=13, out_features=32, bias=True)
  (1): Tanh()
  (2): Linear(in_features=32, out_features=64, bias=True)
  (3): Tanh()
  (4): Linear(in_features=64, out_features=16, bias=True)
  (5): Tanh()
  (6): Linear(in_features=16, out_features=1, bias=True)
)

In [102]:
optimizer2 = optim.SGD(seq_model_multiple.parameters(), lr=1e-2)

training_loop(
    n_epochs = 5000,
    optimizer=optimizer2,
    model = seq_model_multiple,
    loss_fn = nn.MSELoss(),
    x_train = x_train_ten,
    x_val = x_val_ten,
    y_train = y_train_ten,
    y_val = y_val_ten
)

Epoch: 1, Training loss: 26469982011392.0000, Validation loss: 25189244993536.0000
Epoch: 250, Training loss: 3644181970944.0000, Validation loss: 2884491542528.0000
Epoch: 500, Training loss: 3644181970944.0000, Validation loss: 2884491542528.0000
Epoch: 750, Training loss: 3644181970944.0000, Validation loss: 2884491542528.0000
Epoch: 1000, Training loss: 3644181970944.0000, Validation loss: 2884491542528.0000
Epoch: 1250, Training loss: 3644181970944.0000, Validation loss: 2884491542528.0000
Epoch: 1500, Training loss: 3644181970944.0000, Validation loss: 2884491542528.0000
Epoch: 1750, Training loss: 3644181970944.0000, Validation loss: 2884491542528.0000
Epoch: 2000, Training loss: 3644181970944.0000, Validation loss: 2884491542528.0000
Epoch: 2250, Training loss: 3644181970944.0000, Validation loss: 2884491542528.0000
Epoch: 2500, Training loss: 3644181970944.0000, Validation loss: 2884491542528.0000
Epoch: 2750, Training loss: 3644181970944.0000, Validation loss: 2884491542528.0